# Baseline Annotation of Observation based on trained spaCy (2.0.18) model 


<fig size= "4">
This notebook provides a baseline annotation suggestion by using refined named entity recognition that we provided using training a deep learning method by prodigy.  </p>
</fig size= "4">  

In [1]:
import spacy 
from spacy import displacy 


In [2]:
from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [37]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)
pd.set_option('display.max_rows', 500)

## Read the Data

In [5]:
importVersion = '013'

In [6]:
path= '../data/01_df_v{0}.pickle'.format(importVersion)# Put the path of the data in your local machine here, consider the letter "r" before the path
dfAstroNova = pd.read_pickle(path)

In [7]:
# Sort the data based on the chapters of the book 
dfAstroNova['chapter'] = dfAstroNova.chapter.replace("appendix b",np.nan).astype(float)  
dfAstroNova = dfAstroNova.rename_axis('MyIdx').sort_values(by = ['chapter', 'MyIdx'], ascending = [True, True])
dfAstroNova.chapter.fillna('appendix b', inplace=True)

In [8]:
dfAstroNova.reset_index(inplace=True)
dfAstroNova=dfAstroNova.drop("MyIdx",axis=1,inplace=False)
dfAstroNova=dfAstroNova.drop("html",axis=1)

In [9]:
type(dfAstroNova)

pandas.core.frame.DataFrame

In [10]:
dfAstroNova.head()

,text,links,italic,chapter,graphic,table,marginal,sentences,tagged
0,Chapter 1,[],[],1,[],[],[],[Chapter 1],"[[(Chapter, None), (1, NUM)]]"
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",[],[],1,[],[],[],"[On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.]","[[(On, None), (the, None), (distinction, None), (between, None), (the, None), (first, None), (motion, None), (and, None), (the, None), (second, None), (or, None), (proper, None), (motions, None), (and, None), (in, None), (the, None), (proper, None), (motio..."
2,"The testimony of the ages confirms that the motions of the planets are orbicular. Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles. For among figures it is circles, and among bodies the heavens, ...",[],[],1,[],[],"[ Terms: 1. The first motion is that of the whole heaven and of all its stars from the east through the lowest part of the heavens to the east, in the period of 24 hours; in the present diagram, ABCD. 2. The second motions are those of the individual plane...","[The testimony of the ages confirms that the motions of the planets are orbicular., Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles., For among figures it is circles, and among bodies the heavens...","[[(The, None), (testimony, None), (of, None), (the, None), (ages, None), (confirms, None), (that, None), (the, None), (motions, None), (of, None), (the, None), (planets, None), (are, None), (orbicular, None)], [(Reason, None), (having, None), (borrowed, No..."
3,"It is just this from which astronomy arose among humans. Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles whe...",[],[],1,[ ch 1 gr 1],[],[],"[It is just this from which astronomy arose among humans., Astronomy's aim is considered to be to show why the stars' motions appear to be irregular on earth, despite their being exceedingly well ordered in heaven, and to investigate the specific circles w...","[[(It, None), (is, None), (just, None), (this, None), (from, None), (which, None), (astronomy, None), (arose, None), (among, None), (humans, None)], [(Astronomy, None), (s, None), (aim, None), (is, None), (considered, None), (to, None), (be, None), (to, No..."
4,"Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles. These were, however, entwined on...",[],[(such],1,[],[],[ 2],"[Before the distinction between the first motion(1) and the second motions(2) was established, people noted (in contemplating the sun, moon and stars) that their diurnal paths were visually very nearly equivalent to circles., These were, however, entwined ...","[[(Before, None), (the, None), (distinction, None), (between, None), (the, None), (first, None), (motion, None), (1, NUM), (and, None), (the, None), (second, None), (motions, None), (2, NUM), (was, None), (established, None), (people, None), (noted, None),..."


In [11]:
df=dfAstroNova.reset_index().text.str.split('.',expand=True).stack().reset_index(level=-1,drop=True)
type(df)

pandas.core.series.Series

In [12]:
 # Create lists to fill with values
l_col1 = []
l_col2 = []

# iterrate over each row and fill our lists
for ix, row in dfAstroNova.iterrows():
    for value in row['sentences']:
        l_col1.append(value)
        l_col2.append(row['chapter'])

# Create new dataframe from the two lists
df= pd.DataFrame({'sentences': l_col1 ,
                         'chapter': l_col2 })
df=df.rename(columns={"sentences":"sents"});

In [13]:
len(df)

6699

In [14]:
df.head()

,sents,chapter
0,Chapter 1,1
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1


# Upload customized spaCy model provioded by our training data in prodigy

In [15]:
path= '../data/Model_U08'
nlp=spacy.load(path)

In [16]:
def entity(x):
    a=[]
    b=[]
    for ent in nlp(x).ents:
        a +=ent.text,
        b +=ent.label_,
    c=list(zip(a,b))
    return c

In [17]:
def noun_chunk(x):
    a=[]
    for chunk in nlp(x).noun_chunks:
        a.append(chunk)
    return a

In [18]:
df['entities'] = df['sents'].apply(lambda x: entity(x))

In [19]:
df['chunks'] = df['sents'].apply(lambda x: noun_chunk(x))

In [20]:
df.head()

,sents,chapter,entities,chunks
0,Chapter 1,1,[],[(Chapter)]
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,[],"[(the, distinction), (the, first, motion), (the, second, or, proper, motions), (the, proper, motions), (the, second, inequality)]"
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1,[],"[(The, testimony), (the, ages), (the, motions), (the, planets)]"
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1,[],"[(Reason), (experience), (their, gyrations), (perfect, circles)]"
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1,[],"[(figures), (it), (circles), (bodies)]"


In [21]:
df.tail()

,sents,chapter,entities,chunks
6694,"To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".",appendix b,"[(Mars, PLAN), (opposition, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (17 hours 20 minutes, TIME), (Mars, PLAN), (eastward, ASTR), (16' 20"", LONG), (Sun, PLAN), (42' 18"", LONG)]","[(the, position), (Mars), (opposition), (Kepler), (the, angular, distance), (Mars), (Earth, —), (the, place), (the, Sun, —), (17, hours), (Mars), (the, Sun)]"
6695,"Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).",appendix b,"[(longitude, ASTR), (Mars, PLAN), (opposition, ASTR), (198° 37' 50"", LONG), (Mars, PLAN), (orbit, ASTR), (198° 37' 10"", LONG), (18° 37' 10"" Libra, LONG)]","[(Kepler), (the, longitude), (Mars), (opposition), (he), (order), (Mars, 's, orbit), (he), (', 10, "", Libra)]"
6696,The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars.,appendix b,"[(Sun, PLAN), (longitude, ASTR), (observation, ASTR), (Mars, PLAN)]","[(The, Sun), (its, longitude), (the, time), (observation), (its, position), (Mars)]"
6697,"Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made.",appendix b,"[(opposition, ASTR), (17 hours 20 minutes, TIME), (March 29, DATE), (observation, ASTR)]","[(the, time), (opposition), (17, hours), (March), (the, observation)]"
6698,"Kepler determined the time of opposition on March 28, 4h 23m AM, old style.",appendix b,"[(opposition, ASTR), (March 28, DATE), (4h 23m, TIME)]","[(Kepler), (the, time), (opposition), (March)]"


In [22]:
def annot_observation_01(x):
    list_en=[]
    for idx in range(0,len(x)):
        list_en.append(x[idx][1])
    if any (lst in ("LONG","DATE","TIME") for lst in list_en):
        y=1
    else:
        y=0 
    return y
                    
       
                   

    

In [23]:
def annot_observation_02(x):
    list_en=[]
    for idx in range(0,len(x)):
        list_en.append(x[idx][1])
    result= all(lst in list_en for lst in ["LONG","DATE","TIME"])
    if result:
        y=1
    else:
        y=0
    return y
              

In [24]:
df['label'] = df['entities'].apply(lambda x: annot_observation_02(x))

In [25]:
df.head()

,sents,chapter,entities,chunks,label
0,Chapter 1,1,[],[(Chapter)],0
1,"On the distinction between the first motion and the second or proper motions; and in the proper motions, between the first and the second inequality.",1,[],"[(the, distinction), (the, first, motion), (the, second, or, proper, motions), (the, proper, motions), (the, second, inequality)]",0
2,The testimony of the ages confirms that the motions of the planets are orbicular.,1,[],"[(The, testimony), (the, ages), (the, motions), (the, planets)]",0
3,"Reason, having borrowed from experience, immediately presumes this: that their gyrations are perfect circles.",1,[],"[(Reason), (experience), (their, gyrations), (perfect, circles)]",0
4,"For among figures it is circles, and among bodies the heavens, that are considered the most perfect.",1,[],"[(figures), (it), (circles), (bodies)]",0


In [26]:
df.tail(10)

,sents,chapter,entities,chunks,label
6689,"Kepler determined the diurnal motions of the Sun and Mars to be 58' 38"" and 22' 36"" respectively, and the sum of the diurnal motions was 1° 21' 14"".",appendix b,"[(diurnal, ASTR), (Sun, PLAN), (Mars, PLAN), (58' 38"", LONG), (22' 36"", LONG), (diurnal, ASTR), (1° 21' 14"", LONG)]","[(Kepler), (the, diurnal, motions), (the, Sun), (Mars), (the, sum), (the, diurnal, motions), (1, °)]",0
6690,"With the sum of the diurnal motions of Mars and the Sun (1° 21' 14""), Kepler computed the time it takes for Mars and the Sun to move 58' 38"", that is, about 17 hours 20 minutes.",appendix b,"[(diurnal, ASTR), (Mars, PLAN), (Sun, PLAN), (1° 21' 14"", LONG), (Mars, PLAN), (Sun, PLAN), (58' 38"", LONG), (17 hours 20 minutes, TIME)]","[(the, sum), (the, diurnal, motions), (Mars), (the, Sun), ((, 1, °), (Kepler), (it), (Mars), (the, Sun)]",0
6691,"As shown in Figure 2B, the Sun is assumed to be at the center of the planetary system.",appendix b,"[(Sun, PLAN)]","[(Figure, 2B), (the, Sun), (the, center), (the, planetary, system)]",0
6692,"The longitudes of Mars and Earth's in reference to the center of the Sun are 198° 21' 30"" and 199° 20' 8"" respectively.",appendix b,"[(longitudes, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (198° 21' 30"", LONG), (199° 20' 8"", LONG)]","[(The, longitudes), (Mars), (Earth), (reference), (the, center), (the, Sun)]",0
6693,"The deviation of elongation between Mars and the Earth remain 58' 38"".",appendix b,"[(elongation, ASTR), (Mars, PLAN), (Earth, PLAN), (58' 38"", LONG)]","[(The, deviation), (elongation), (Mars), (the, Earth)]",0
6694,"To find the position of Mars at opposition, Kepler computed the angular distance that Mars and Earth—now substituting the place of the Sun—moved during 17 hours 20 minutes; Mars moved eastward about 16' 20"" and the Sun westward about 42' 18"".",appendix b,"[(Mars, PLAN), (opposition, ASTR), (Mars, PLAN), (Earth, PLAN), (Sun, PLAN), (17 hours 20 minutes, TIME), (Mars, PLAN), (eastward, ASTR), (16' 20"", LONG), (Sun, PLAN), (42' 18"", LONG)]","[(the, position), (Mars), (opposition), (Kepler), (the, angular, distance), (Mars), (Earth, —), (the, place), (the, Sun, —), (17, hours), (Mars), (the, Sun)]",0
6695,"Accordingly, Kepler determined the longitude of Mars at opposition to be 198° 37' 50"" from which he subtracted about 39"" in order to correct Mars's orbit; he got 198° 37' 10"" (18° 37' 10"" Libra).",appendix b,"[(longitude, ASTR), (Mars, PLAN), (opposition, ASTR), (198° 37' 50"", LONG), (Mars, PLAN), (orbit, ASTR), (198° 37' 10"", LONG), (18° 37' 10"" Libra, LONG)]","[(Kepler), (the, longitude), (Mars), (opposition), (he), (order), (Mars, 's, orbit), (he), (', 10, "", Libra)]",0
6696,The Sun moved westward and its longitude decreased from the time of observation to its position opposite to Mars.,appendix b,"[(Sun, PLAN), (longitude, ASTR), (observation, ASTR), (Mars, PLAN)]","[(The, Sun), (its, longitude), (the, time), (observation), (its, position), (Mars)]",0
6697,"Therefore, the time of opposition is 17 hours 20 minutes before March 29, at 21:43, the time when the observation was made.",appendix b,"[(opposition, ASTR), (17 hours 20 minutes, TIME), (March 29, DATE), (observation, ASTR)]","[(the, time), (opposition), (17, hours), (March), (the, observation)]",0
6698,"Kepler determined the time of opposition on March 28, 4h 23m AM, old style.",appendix b,"[(opposition, ASTR), (March 28, DATE), (4h 23m, TIME)]","[(Kepler), (the, time), (opposition), (March)]",0


In [27]:
df[df.chapter==10]


,sents,chapter,entities,chunks,label
962,Chapter 10,10,[],[(Chapter)],0
963,"Consideration of the observations themselves, with which Tycho Brahe hunted for the moments of opposition to the mean sun.",10,"[(Tycho Brahe, NAME), (opposition, ASTR)]","[(Consideration), (the, observations), (themselves), (Tycho, Brahe), (the, moments), (opposition), (the, mean, sun)]",0
964,"In an enquiry of such precision, I could not have foregone a deeper inspection of the foundations themselves.",10,[],"[(an, enquiry), (such, precision), (I), (a, deeper, inspection), (the, foundations), (themselves)]",0
965,And Brahe had given me the opportunity to make use of his observations.,10,[],"[(Brahe), (me), (the, opportunity), (use), (his, observations)]",0
966,This is what I found.,10,[],"[(what), (I)]",0
...,...,...,...,...,...
1030,"However, even in observations at the island it too often happened that right ascensions measured from two different stars differed by three minutes.",10,[],"[(observations), (the, island), (it), (right, ascensions), (two, different, stars), (three, minutes)]",0
1031,"When I asked Christian [Longomontanus], on this subject, whether I should consider this an effect of the limitations of observation or vision, he replied, “This is not unusual.”",10,"[(observation, ASTR)]","[(I), (Christian, [, Longomontanus), (this, subject), (I), (the, limitations), (observation), (vision), (he)]",0
1032,"Finally, the reader should be advised that Tycho, in his table, claimed to have made use of the solar parallax in correcting the positions of Mars.",10,"[(solar, ASTR), (parallax, ASTR), (Mars, PLAN)]","[(the, reader), (Tycho), (his, table), (use), (the, solar, parallax), (the, positions), (Mars)]",0
1033,But it will now shortly be made clear that the parallax of Mars is a slippery and imperceptible business.,10,"[(parallax, ASTR), (Mars, PLAN)]","[(it), (the, parallax), (Mars), (a, slippery, and, imperceptible, business)]",0


In [38]:
df[df.chapter==]


,sents,chapter,entities,chunks,label
1035,Chapter 11,11,[],[(Chapter)],0
1036,On the diurnal parallaxes of the star Mars.,11,"[(diurnal, ASTR), (star, ASTR), (Mars, PLAN)]","[(the, diurnal, parallaxes), (the, star, Mars)]",0
1037,The starting point of my new elaboration and renewal of the motions was where I have just stopped.,11,[],"[(The, starting, point), (my, new, elaboration), (renewal), (the, motions), (I)]",0
1038,For it is clear from Part I that the positions of Mars should be taken at the moments of true opposition to the sun.,11,"[(Mars, PLAN), (opposition, ASTR)]","[(it), (Part), (I), (the, positions), (Mars), (the, moments), (true, opposition), (the, sun)]",0
1039,"However, it is also clear that not every trace of the second inequality is thus removed, it being also necessary to refer the arc measured on the ecliptic to the planet’s orbit.",11,"[(ecliptic, ASTR), (orbit, ASTR)]","[(it), (not, every, trace), (the, second, inequality), (it), (the, arc), (the, ecliptic), (the, planet, ’s, orbit)]",0
1040,But the planet’s orbit must first be investigated by the inclination of the planes and by finding out the nodes.,11,"[(orbit, ASTR), (inclination, ASTR), (nodes, ASTR)]","[(the, planet, ’s, orbit), (the, inclination), (the, planes), (the, nodes)]",0
1041,"Again, the inclination and the nodes cannot be found without knowing the diurnal parallax, at least if this should turn out to be relatively large.",11,"[(inclination, ASTR), (nodes, ASTR), (diurnal, ASTR), (parallax, ASTR)]","[(the, inclination), (the, nodes), (the, diurnal, parallax)]",0
1042,One must therefore start with the parallax.,11,"[(parallax, ASTR)]","[(One), (the, parallax)]",0
1043,I shall present two ways to find it.,11,[],"[(I), (two, ways), (it)]",0
1044,The first way (and the one more familiar to others) will be examined using the Brahean observations.,11,"[(Brahean, NAME)]","[(The, first, way), (the, one), (others), (the, Brahean, observations)]",0


In [54]:
df[(df.label==1)]

,sents,chapter,entities,chunks,label
967,"On 1580 November 12 at 10h 50m,1 they set Mars down at 8° 36’ 50” Gemini2 without mentioning the horizontal variations, by which term I wish the diurnal parallaxes and the refractions to be understood in what follows.",10,"[(1580 November 12, DATE), (10h, TIME), (Mars, PLAN), (8° 36’ 50” Gemini2, LONG), (diurnal, ASTR)]","[(November), (they), (Mars), (8, °), (Gemini2), (the, horizontal, variations), (which, term), (I), (the, diurnal, parallaxes), (the, refractions), (what)]",1
977,"On 1582 December 28 at 11h 30m, they set Mars down at 16° 47’ Cancer by observation ^6.",10,"[(1582 December 28, DATE), (11h 30m, TIME), (Mars, PLAN), (16° 47’ Cancer, LONG), (observation, ASTR)]","[(December), (11h), (they), (Mars), (16, °, 47, ’, Cancer)]",1
984,"On 1585 January 31 at 12h 0m, Mars was placed at 21° 18’ 11” Leo ^8.",10,"[(1585 January 31, DATE), (12h 0m, TIME), (Mars, PLAN), (21° 18’ 11” Leo, LONG)]","[(January), (12h, 0, m), (Mars)]",1
992,"On 1587 March 7 at 19h 10m they deduced the position of Mars from the observations, which was 25° 10’ 20” Virgo.",10,"[(1587 March 7, DATE), (19h 10m, TIME), (Mars, PLAN), (25° 10’ 20” Virgo., LONG)]","[(March), (they), (the, position), (Mars), (the, observations), (Virgo)]",1
997,"On 1589 April 15 at 12h 5m they established the position of Mars very carefully at 3° 58' 21"" Scorpio, and corrected for longitudinal parallax so as to make it 3° 57’ 11” ^10.",10,"[(1589 April 15, DATE), (12h 5m, TIME), (Mars, PLAN), (3° 58' 21"" Scorpio, LONG), (longitudinal, ASTR), (parallax, ASTR), (3° 57’ 11”, LONG)]","[(April), (12h), (they), (the, position), (Mars), (3, °, 58, ', 21, "", Scorpio), (longitudinal, parallax), (it)]",1
1001,"On 1591 June 6 at 12h 20m, Mars is placed at 27° 15’ Sagittarius ^11.",10,"[(1591 June 6, DATE), (12h 20m, TIME), (Mars, PLAN), (27° 15’ Sagittarius, LONG)]","[(June), (20, m), (Mars)]",1
1012,"On 1595 October 30 at 8h 20m, they found Mars at 17° 48’ Taurus, with a diurnal motion of 22’ 54” ^15.",10,"[(1595 October 30, DATE), (8h 20m, TIME), (Mars, PLAN), (17° 48’ Taurus, LONG), (diurnal, ASTR), (22’ 54”, LONG)]","[(October), (they), (Mars), (Taurus), (a, diurnal, motion)]",1
1016,"On 1597 December 10 at 8h 30m, they first placed Mars at 3° 30’ Cancer, and again at 4° 1’ Cancer, the mean being 3° 45½’ Cancer ^16.",10,"[(1597 December 10, DATE), (8h 30m, TIME), (Mars, PLAN), (3° 30’ Cancer, LONG), (4° 1’ Cancer, LONG)]","[(December, 10, at, 8h, 30, m), (they), (Mars), (3, °, 30, ’, Cancer), (4, °, 1, ’, Cancer), (Cancer)]",1
1027,"This gives a diurnal motion of 23’ 44”, and a position on January 19/29 at 9h 40m of 8° 18’ 45” Leo, just as they put it.",10,"[(diurnal, ASTR), (23’ 44”, LONG), (January 19/29, DATE), (9h 40m, TIME), (8° 18’ 45” Leo, LONG)]","[(a, diurnal, motion), (a, position), (January), (Leo), (they), (it)]",1
1071,"Again, at 7h 15m on the morning of December 27, it was 36° 43’ from Cor Leonis,7 whose latitude is 0° 26½’; hence, its longitude at the end of 1582 is 17° 28⅓’ Cancer, altitude 14° 4’, and thus affected by refraction.",11,"[(7h 15m, TIME), (December 27, DATE), (36° 43’, LONG), (Cor Leonis,7, STAR), (latitude, ASTR), (longitude, ASTR), (altitude, ASTR), (14° 4’, LONG), (refraction, ASTR)]","[(15, m), (the, morning), (December), (it), (Cor, Leonis,7), (whose, latitude), (its, longitude), (the, end), (’, Cancer), (refraction)]",1


In [ ]:
def show_chapter(x):
     return df.loc[df.chapter==10]



In [ ]:
len(df)

In [ ]:
interact(show_chapter,x=(1,70,1))
         

## Save the result as a JSON file

Since we used spacy in columns and spacy used cython, converting directly to json was not possible. we did this approach to save the data to json. we Have saved the data with the different version and annotation policy. 

In [ ]:
df.to_csv("./df_obs.csv")

In [ ]:
csvFile = pd.DataFrame(pd.read_csv("df_obs.csv", sep = ",", header = 0, index_col = False))
csvFile.to_json("dfObs_a02_v02.json")

In [ ]:
pd.read_json('dfObs_a02_v02.json', sorted);
df.sort_index(axis=0);